In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd

### 제주도 농어촌민박(펜션, 게스트하우스, 민박 등)
- 출처 - 제주데이터허브
- https://jejudatahub.net/data/view/data/956


In [3]:
df_accomo = pd.read_csv('./data/제주농어촌민박현황.csv')
df_accomo.head(20)

,읍면동,민박명,소재지,데이터기준일
0,한림읍,기영 민박,제주시 한림읍 한림로 345,20211123
1,한림읍,에너벨리,제주시 한림읍 한림로 356,20211123
2,한림읍,협재 빌리지,제주시 한림읍 한림로 338-1,20211123
3,한림읍,해송민박,제주시 한림읍 협재4길 7-15,20211123
4,한림읍,영심 민박,제주시 한림읍 협재4길 7-7,20211123
5,한림읍,원룸민박,제주시 한림읍 비양도길 275,20211123
6,한림읍,바다그리기,제주시 한림읍 옹포2길 16,20211123
7,한림읍,비양도 민박,제주시 한림읍 비양도길 8,20211123
8,한림읍,해조대 민박,제주시 한림읍 한림로 339,20211123
9,한림읍,이쁜새민박,제주시 한림읍 협재로 68-5,20211123


In [4]:
accomo_list = df_accomo['민박명']
accomo_list

0              기영 민박
1               에너벨리
2             협재 빌리지
3               해송민박
4              영심 민박
            ...     
2979         화랑펜션2층집
2980    김군빌리지 게스트하우스
2981           도두봉팬션
2982           바다성펜션
2983             올레뷰
Name: 민박명, Length: 2984, dtype: object

In [5]:
accomo_list1 = df_accomo[df_accomo['읍면동'] == '한림읍']['민박명']
accomo_list1

0       기영 민박
1        에너벨리
2      협재 빌리지
3        해송민박
4       영심 민박
        ...  
459    포인트 무드
460    스테이 담담
461        오닐
462     호미하우스
463       코티지
Name: 민박명, Length: 464, dtype: object

### 네이버 블로그 데이터 확인



In [12]:
# 제주 + "제주낭만펜션" 검색결과 URL
# "" 포함시 필수검색 옵션
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+%22%EB%82%AD%EB%A7%8C%EC%A0%9C%EC%A3%BC%ED%8E%9C%EC%85%98%22+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C&tqi'

In [ ]:
result = []
cnt = 0

for accomo in tqdm(accomo_list):

    # 숙소 이름별 - 검색
    base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
    accomo_name = quote(f'"{accomo}"')

    # 제외하고자 하는 검색어
    # 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
    # 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
    delete_words = ["파트너스", "예약코드", '부동산']
    delete_words

    delete_search = ""
    for word in delete_words:    
        delete_search += f'+-{quote(word)}'

    # 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
    url = base_url + accomo_name + delete_search
    driver = webdriver.Chrome()
    driver.get(url)


    # 스크롤 내리기

    # len(soup.select('.detail_box')) = 30 첫페이지에 나오는 수가 30개
    # 30개 이하인 경우 분석에서 제외
    if len(driver.find_elements(By.CLASS_NAME, 'detail_box')) > 30:
        continue
    
    else :
        # 스크롤은 3번만 내리자
        for _ in range(5):
            last_height = driver.execute_script("return document.body.scrollHeight")

            # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
            driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
            time.sleep(round(random.uniform(0.0, 1.0), 3))
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
            time.sleep(round(random.uniform(0.0, 1.2), 3))

            # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
    
    time.sleep(round(random.uniform(0.0, 2.2), 3))
    try :        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        lis = soup.select_one('.lst_view').select('.view_wrap')
        
        title_list = []
        content_list = []
        blog_list = []
        time_list = []

        # 블로그가 10개 미만인 경우 분석에서 제외
        if len(lis) >= 10:
            for li in lis:
                try :
                    title_raw = li.select_one('.title_area').select_one('a').text.strip()
                    #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
                    title_list.append(title_raw)
                except:
                    title_list = 'error'
                
                try :
                    content_raw = li.select_one('.dsc_link').text.strip()
                    #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
                    content_list.append(content_raw)  
                except :
                    content_list = 'error'
                try :          
                    blog_link = li.select_one('.title_area').select_one('a')['href']
                    blog_list.append(blog_link)
                except :
                    blog_list = 'error'

                try :          
                    blog_time = li.select_one('.sub').text.strip()
                    time_list.append(blog_time)
                except :
                    time_list = 'error'
    except :
        title_list = []
        content_list = []
        blog_list = []
        time_list = []


    result.append({'name' : accomo,
                   'title' : title_list,
                   'contents' : content_list,
                   'time' : time_list,
                   'blog' : blog_list})
    cnt += 1
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    print(f'{cnt}번째 "{accomo}" 블로그 링크 수집완료')

time.sleep(round(random.uniform(0.0, 4.2), 3))



In [ ]:
df1 = pd.DataFrame(result)
df1['blog_num'] = df1['blog'].apply(lambda x: len(x))

df1.to_csv('./blog_list.csv')

In [86]:
df = pd.read_csv('./bigfile/blog_list.csv')
df = df[[ 'name', 'title', 'contents', 'time', 'blog', 'blog_num']]
df

,name,title,contents,time,blog,blog_num
0,기영 민박,"['[제주여행] 제주 마지막날 Day3 / 바다에서 하염없이 멍때리기...', '제...",['옮겨 #기영민박 들려서 맥주 병나발 불고 육지로 돌아가기 싫음을 온몸으로 표현 ...,"['2023.06.05.', '2023.11.26.', '2016.12.13.', ...",['https://blog.naver.com/imvaney/223121121422'...,90
1,에너벨리,"['해수욕장(동쪽주차장), 협재 에너벨리(식당), 카페노티드제주', '몰아 올리는 ...",['제주 #협재에너벨리 - 주소 : 제주 제주시 한림읍 한림로 356 에너벨리식당&...,"['2022.08.25.', '2023.06.14.', '2주 전', '2023.0...",['https://blog.naver.com/hi_7312319/2228582276...,180
2,협재 빌리지,"['2021.11.07(일)트랜스제주50 후 백패킹 금능해수욕장', '제주협재해수욕...",['옴기고 협재빌리지모텔민박 064- 796-4165 5만원에 방잡음 덴트 철수하러...,"['2021.11.09.', '2017.10.27.', '2016.01.13.', ...","['https://blog.naver.com/mowcho/222563341734',...",15
3,해송민박,['[19.03.02 ~ 05] DAY3~6 - 퇴사 여행. 8박 9일 제주 자유여...,['저녁 먹으러 주변에 나가볼까 했는데 죄다 브레이크타임 걸려서 포기 바람 개오졌던...,"['2019.05.20.', '2017.03.05.', '2009.06.14.', ...",['https://blog.naver.com/e__uni_/221542178596'...,18
4,영심 민박,[],[],[],[],0
...,...,...,...,...,...,...
2979,화랑펜션2층집,[],[],[],[],0
2980,김군빌리지 게스트하우스,"['7월21일~30일 제주도여름휴가... 그랜드조선제주 루브리카', '제주 공항 파...",['놀 논짓물로 이동하여 주차장에 주차 후 취침했습니다~ 2023년 제주도 여름휴가...,"['2023.08.01.', '2023.09.15.', '2023.09.14.', ...",['https://blog.naver.com/bong1248/223171542190...,180
2981,도두봉팬션,[],[],[],[],0
2982,바다성펜션,"['제주 해변 걷기 좋은 외도 연대포구 바다 풍경', '22년의 여름휴가(제주1)'...","['제주베이, 바다성펜션, 가까이에 파도소리, 별하바나펜션, 하늘타리펜션, 섬타이즈...","['2023.10.06.', '2022.09.05.', '2015.02.23.', ...",['https://blog.naver.com/face8901/223229815097...,42


In [87]:
df[df['contents'] == 'error']
# 149개의 숙박시설에 관한 컨텐츠가 모아지지 않았다.
# 이유는 못찼겠어요. - 시간관계상 포기

,name,title,contents,time,blog,blog_num
122,소희네 하우스,"['[2023.02.17 ~ 2023.02.19] 웃기만 했던 제주 여행기', '향...",error,"['2023.02.21.', '2023.07.17.', '2023.02.28.', ...",['https://blog.naver.com/su_in1025/22302285395...,180
137,바다아리,"['7개월아기랑 제주여행 1-2일차... 해물한상, 제주 이니스프리...', '아이...",error,"['2023.04.14.', '2022.07.24.', '2022.07.09.', ...",['https://blog.naver.com/happymay525/223074988...,180
144,제주리조텔,"['(애월)빌레리조트+(협재)제주리조텔+(서귀포)이스턴제주호...', '제주 10일...",error,"['2022.12.05.', '2023.06.10.', '2021.02.26.', ...",['https://blog.naver.com/yeom8266/222946389243...,109
160,집 머무는,"['유디 in 제주 - 둘 째 날♀️', '[jeju 신혼여행 4일차] 제주 / 집...",error,"['2023.06.22.', '2023.06.30.', '2023.11.01.', ...",['https://blog.naver.com/uzi0120/223135621722'...,131
193,협재45m,"['220619 제주여행', '[제주/협재/독채펜션] 협재45m', ""제주 협재 독...",error,"['2022.06.20.', '2022.04.03.', '2022.08.11.', ...",['https://blog.naver.com/pado-chida/2227812395...,180
...,...,...,...,...,...,...
2928,데이드림,"['제주일지 2편', '[수영복리뷰]졸린 잭슨 데이드림/제주 해수욕장 추천(금능.....",error,"['2023.03.13.', '2023.08.31.', '2022.03.04.', ...",['https://blog.naver.com/tldlsdmswl/2230425929...,180
2934,제주시크릿가든,"['제주 씨에스(seaes) 호텔&리조트-제주스러움을 만끽할 수...', '제주커플...",error,"['2023.01.27.', '2023.09.15.', '2023.08.02.', ...",['https://blog.naver.com/kminju611/22299677574...,80
2962,까르마,"['제주 독채 펜션 힐링거점이었던 풀빌라 까르마', '제주 풀빌라 미온수풀장있는 제...",error,"['2023.06.01.', '2023.01.16.', '2023.11.02.', ...","['https://blog.naver.com/enzywu/223117917995',...",180
2964,낭만제주펜션,"['종달리 숙소 :: 낭만제주펜션 / 제주도 감성 숙소 / 제주서쪽숙소', '제주도...",error,"['2022.02.22.', '2022.10.12.', '2023.01.05.', ...",['https://blog.naver.com/honey_lulu/2226550712...,149


In [88]:
# 수집이 리스트안의 리스트가 아닌 각 글자로 들어간듯....
df['time'][0]

"['2023.06.05.', '2023.11.26.', '2016.12.13.', '2021.01.04.', '2021.07.03.', '2015.12.30.', '2018.01.02.', '2015.09.24.', '2016.08.27.', '2016.08.21.', '2015.06.27.', '2015.04.05.', '2016.08.27.', '2016.12.18.', '2015.08.01.', '2017.09.03.', '2019.11.27.', '2016.05.18.', '2016.08.12.', '2016.01.09.', '2016.10.19.', '2016.06.06.', '2016.05.27.', '2016.09.02.', '2016.09.28.', '2016.06.19.', '2015.08.15.', '2016.04.25.', '2014.09.14.', '2015.01.06.', '2016.07.22.', '2014.06.04.', '2016.04.15.', '2016.09.24.', '2023.06.22.', '2015.08.14.', '2017.10.01.', '2016.04.25.', '2016.01.25.', '2016.08.10.', '2016.01.18.', '2016.07.22.', '2016.09.02.', '2015.11.03.', '2016.06.02.', '2016.06.02.', '2021.01.05.', '2017.06.06.', '2015.08.11.', '2015.08.16.', '2015.08.16.', '2015.08.03.', '2015.09.11.', '2015.09.11.', '2015.08.01.', '2015.09.21.', '2016.01.26.', '2016.01.26.', '2018.07.23.', '2015.07.31.', '2016.07.22.', '2014.06.04.', '2016.04.15.', '2016.09.24.', '2023.06.22.', '2015.08.14.', '2017.10

In [89]:
# time 특수문자 제거후에 ,로 나누기
df['time'] = df['time'].apply(lambda x: re.sub("[\[\]''']","",x).split(','))
df['blog'] = df['blog'].apply(lambda x: re.sub("[\[\]]","",x).split(','))

In [90]:
temp = df[df['blog_num'] > 0]
len(temp)

1580

In [81]:
def convert_date(s):
    if len(s) == 11 : # 2023.12.13.  
        return s[:-1] # 마지막 점 제거
    
    now = datetime.now()
    if '어제' in s:
        date = now - timedelta(days = 1)
    elif '일' in s:
        date = now - timedelta(days = int(s[0]))
    elif '시간' in s:
        date = now - timedelta(hours = int(s[:-4]))
    elif '분' in s:
        date = now - timedelta(minutes = int(s[:-3]))
    elif '주' in s:
        date = now - timedelta(weeks = int(s[:-3]))

    return date.strftime('%Y.%m.%d')

# date = now - timedelta(weeks=int(re.sub(r'[^0-9]',"",s)))

In [102]:
accomo_contents = []

for i in range(1, 2):
    temp2 = temp.iloc[i,]
    for j in range(len(temp.iloc[i,]['time'])):
        naver_time = temp2['time'][j].strip()
        new_time = convert_date(naver_time)
        blog_url = temp2['blog'][j].strip().replace("'","") 
        print(blog_url)

        driver = webdriver.Chrome()
        driver.get(blog_url)


    time.sleep(round(random.uniform(0.0, 4.2), 3))
        


https://blog.naver.com/hi_7312319/222858227603
https://blog.naver.com/ysunnylee/223129124265
https://blog.naver.com/hshhsh0820/223304215746


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.200)
Stacktrace:
	GetHandleVerifier [0x00007FF62C6F2142+3514994]
	(No symbol) [0x00007FF62C310CE2]
	(No symbol) [0x00007FF62C1B76AA]
	(No symbol) [0x00007FF62C190AFD]
	(No symbol) [0x00007FF62C22CB1B]
	(No symbol) [0x00007FF62C24218F]
	(No symbol) [0x00007FF62C225D93]
	(No symbol) [0x00007FF62C1F4BDC]
	(No symbol) [0x00007FF62C1F5C64]
	GetHandleVerifier [0x00007FF62C71E16B+3695259]
	GetHandleVerifier [0x00007FF62C776737+4057191]
	GetHandleVerifier [0x00007FF62C76E4E3+4023827]
	GetHandleVerifier [0x00007FF62C4404F9+689705]
	(No symbol) [0x00007FF62C31C048]
	(No symbol) [0x00007FF62C318044]
	(No symbol) [0x00007FF62C3181C9]
	(No symbol) [0x00007FF62C3088C4]
	BaseThreadInitThunk [0x00007FFD95A47344+20]
	RtlUserThreadStart [0x00007FFD95B826B1+33]


In [107]:
accomo_contents = []


for i in range(1, 2):
    temp2 = temp.iloc[i,]
    for j in range(len(temp.iloc[i,]['time'])):
        naver_time = temp2['time'][j].strip()
        new_time = convert_date(naver_time)        
        blog_url = temp2['blog'][j].strip().replace("'","")
        cnt = 0
        
        if new_time > '2019.12.31':                     
        
            if blog_url.find('blog.naver.com') > 0:
                        
                driver = webdriver.Chrome()
                driver.get(blog_url)
                driver.switch_to.frame('mainFrame')
                time.sleep(round(random.uniform(0.0, 1.2), 3))

                
                lis = driver.find_element(By.CLASS_NAME, 'se-main-container').find_elements(By.CLASS_NAME, 'se-text')
                blog_contents = []

                for li in lis:
                    content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())
                    blog_contents.append(content.replace('\U0001f979','').replace('\n',''))
                    text = ' '.join(blog_contents)

                cnt += 1
                accomo_contents.append(blog_contents)
                time.sleep(round(random.uniform(0.0, 1.2), 3))
                    
                

                if cnt > 30:
                        break
        time.sleep(round(random.uniform(0.0, 4.2), 3))



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".se-main-container"}
  (Session info: chrome=120.0.6099.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF62C6F2142+3514994]
	(No symbol) [0x00007FF62C310CE2]
	(No symbol) [0x00007FF62C1B76AA]
	(No symbol) [0x00007FF62C201860]
	(No symbol) [0x00007FF62C20197C]
	(No symbol) [0x00007FF62C244EE7]
	(No symbol) [0x00007FF62C22602F]
	(No symbol) [0x00007FF62C2428F6]
	(No symbol) [0x00007FF62C225D93]
	(No symbol) [0x00007FF62C1F4BDC]
	(No symbol) [0x00007FF62C1F5C64]
	GetHandleVerifier [0x00007FF62C71E16B+3695259]
	GetHandleVerifier [0x00007FF62C776737+4057191]
	GetHandleVerifier [0x00007FF62C76E4E3+4023827]
	GetHandleVerifier [0x00007FF62C4404F9+689705]
	(No symbol) [0x00007FF62C31C048]
	(No symbol) [0x00007FF62C318044]
	(No symbol) [0x00007

In [108]:
blog_contents

['',
 '때는 바야흐로 인턴 생활에 울부짖던 2월 초제주 비행기를 끊었다는 굥이의 말에부럽다며 울다가 굥이가 같이 가자고 해서충동같지만 충동 아닌 제주 여행을 계획하게 됐다여행은 3월 9일3월 12일로 3박 4일 이었다그냥 인턴 끝난 기념으로 가볍게 떠난 여행이었는데날씨도 너무 좋고 제주에서 만난 모든 것들이 환상적이라내 인생에서 좋았던 여행 탑에 꼽을 수 있을 것 같다3월 9일',
 '굥이는 서울에서 제주로나는 인턴 후 울산에 내려갔기 때문에울산에서 제주로 갔다때문에 우리 둘은 제주 공항에서 만났다한 달만에 만난 건데한 1년은 못만난 사람들처럼 호들갑 떨었다 원래는 전에 솔희가 추천해줬던숙성도에 가기로 했으나인기 많은 가게라더니 예약이 모두 마감돼서 ㅜㅜ급하게 서치해서 돈오름 숯불갈비 라는 가게에 갔다1일차 저녁  돈오름 숯불갈비',
 '배고픈 상태에 제주 흑돼지라니 맛있을 수 밖에 없었다돈오름 숯불갈비는 본점이 있고 제원점이 있는데본점에서 네이버예약하면 간천엽  을 준다길래 갔는데내가 예약을 제원점으로 잘못해서 결국은 못먹었다맛있긴했는데 강추 이런 느낌은 아니었고 걍 무난했다',
 '내가 블로그 쓸 거라고제주도에서 사진 많이 찍을거라 해서식당 앞에서 사진 찍었는데 굥이가 공기에몸통박치기를멋진 옆돌기를 보여줬다',
 '호텔 돌아가는 길에 본 제주도스러운 것들첫째날 숙소는 공항근처의 저렴한 호텔로 했다어차피 잠만 잘 거라서 ',
 '근데 그런 것 치고 꽤나 멋진 야식을 먹었다영화로 아메리칸셰프 봤는데대선 때문에 속터져서 결국 끝까지 못 보고 껐다',
 '2번 아저씨가 열심히 넥스트레벨 추고 계시는 모습 3월 10일호텔 체크아웃을 하고 애월로 향했다',
 '중간에 환승해야 해서 내렸는데날씨도 너무 좋고 버정 앞의 초등학교가 동화 같았다이런거에 약함  밝게 자라렴 꿈나무들아  갑자기두 번째날 숙소는 글라애월 이었다에어비앤비로 예약했는데사실 기대 안했는데 너무너무 좋았다',
 '둘이 쓰기 넓은 방이라 쾌적했고 따듯해서 좋았다그리고 주인분들이 정말 친절하셨다 ㅠㅠ

In [117]:
driver = webdriver.Chrome()
driver.get(blog_url)
driver.switch_to.frame('mainFrame')



In [123]:
for i in driver.find_element(By.CLASS_NAME, 'se_component').find_elements(By.CLASS_NAME, 'se_textarea'):
    print(i.text)

10월제주여행(우도,삼해인관광호텔,핫플투어-서부감성투어,스완지리조트,도두봉)


In [128]:
for i in driver.find_elements(By.CLASS_NAME, 'se_component'):
    print(i.find_element(By.CLASS_NAME, 'se_textarea').text)

10월제주여행(우도,삼해인관광호텔,핫플투어-서부감성투어,스완지리조트,도두봉)
어쩌다 받게된 3일간의 휴가 
그냥 집에서 쉬기만 하기는 아까워 떠난 2박 3일 제주 여행 

*여행준비*
-항공
티몬 월요일 오전8시 비행기 1인 62280원(김포->제주)
         수요일 오전 10시비행기 1인 28890원(제주->김포)
-숙소
첫째날 삼해인 관광호텔 트윈룸 2인조식포함 56525원
둘째날 스완지리조트 스탠다드트윈룸65500원
-투어버스
핫플투어 서부감성투어 1인29800원

장롱면허 10년차는 뚜벅이 여행을 갑니다
5월에 떠난 일일투어가 너무 좋았어서 2일 동안 일일투어 2개 코스로 떠나고 싶었으나 공항 픽업시간 9시 칼같이 떠나버리는 투어버스 특성상 8시 비행기로는 9시투어를 갈 수 없어요ㅠㅠ

더 빠른 비행기로 가자니 김포공행항 첫차를 타도 아슬아슬 비행기 놓치느니 첫째날은 여유롭게 우도로 자유투어 결정
티웨이항공타고 제주로
제주 도착
오전 8시 비행기를 타고 9시 25분쯤 제주공항 도착 버스타는곳에서 1번홈 남원 표선 성산 방면에서 111번 또는 112번 버스를타면 성산포항까지 1시간20~30분 정도면 도착
111번112번 버스정보
https://bus.jeju.go.kr/schedule/view/111


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".se_textarea"}
  (Session info: chrome=120.0.6099.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF62C6F2142+3514994]
	(No symbol) [0x00007FF62C310CE2]
	(No symbol) [0x00007FF62C1B76AA]
	(No symbol) [0x00007FF62C201860]
	(No symbol) [0x00007FF62C20197C]
	(No symbol) [0x00007FF62C1F64FC]
	(No symbol) [0x00007FF62C22602F]
	(No symbol) [0x00007FF62C1F63B6]
	(No symbol) [0x00007FF62C226490]
	(No symbol) [0x00007FF62C2428F6]
	(No symbol) [0x00007FF62C225D93]
	(No symbol) [0x00007FF62C1F4BDC]
	(No symbol) [0x00007FF62C1F5C64]
	GetHandleVerifier [0x00007FF62C71E16B+3695259]
	GetHandleVerifier [0x00007FF62C776737+4057191]
	GetHandleVerifier [0x00007FF62C76E4E3+4023827]
	GetHandleVerifier [0x00007FF62C4404F9+689705]
	(No symbol) [0x00007FF62C3